In [1]:
import os
from surfer import Brain, project_volume_data
from mayavi import mlab
import matplotlib.pyplot as plt

mlab.init_notebook(backend='png')

Notebook initialized with png backend.


In [2]:
mri_file = os.path.join(os.getcwd(), "../data/mri/reg/avgCBF_thr10_MNI.nii.gz")
reg_file = os.path.join(os.environ["FREESURFER_HOME"], "average/mni152.register.dat")

SIZE = 1200
brain_lh = Brain(subject_id = "fsaverage5", hemi = "lh", surf = "inflated", title = 'Average CBF (thr10)',
              views=['med', 'lat'], background="white", size=SIZE)
brain_rh = Brain(subject_id = "fsaverage5", hemi = "rh", surf = "inflated", title = 'Average CBF (thr10)',
              views=['lat', 'med'], background="white", size=SIZE)

In [3]:

surf_data_lh = project_volume_data(mri_file, "lh", reg_file, smooth_fwhm=3)
surf_data_rh = project_volume_data(mri_file, "rh", reg_file, smooth_fwhm=3)

In [4]:
brain_lh.add_overlay(surf_data_lh, min=0, max=90, name="cbf_lh", hemi='lh', sign="pos")
brain_rh.add_overlay(surf_data_rh, min=0, max=90, name="cbf_rh", hemi='rh', sign="pos")

In [5]:
#brain_lh.hide_colorbar(row=-1, col=-1)
#brain_lh.hide_colorbar(row=-2, col=-2)
brain_lh.save_imageset(prefix="tmp/lh_cbf", views=["med", "lat"], colorbar=None)

#brain_rh.hide_colorbar(row=-2, col=-2)
brain_rh.save_imageset(prefix = "tmp/rh_cbf", ["lat", "med"])

SyntaxError: positional argument follows keyword argument (<ipython-input-5-b68b3ba0be7d>, line 6)

In [14]:
from PIL import Image

im1 = Image.open(os.path.join(os.getcwd(),'tmp/lh_cbf_med.png'))
im2 = Image.open(os.path.join(os.getcwd(),'tmp/lh_cbf_lat.png'))
im3 = Image.open(os.path.join(os.getcwd(),'tmp/rh_cbf_lat.png'))
im4 = Image.open(os.path.join(os.getcwd(),'tmp/rh_cbf_med.png'))

def get_concat_h(im1, im2):
    dst = Image.new('RGB', (im1.width * 4, im1.height))
    dst.paste(im1, (0, 0))
    dst.paste(im2, (im1.width, 0))
    dst.paste(im3, (im1.width*2, 0))
    dst.paste(im4, (im1.width*3, 0))
    return dst

#def concat_grid

get_concat_h(im1, im2).save('tmp/concat.png')

In [15]:
os.getcwd()

'/Users/vgonzenb/PennSIVE/ISLA/notebook'

In [16]:
#brain.save_image('../tmp/surf_avgCBF_thr10.png')